In [ ]:
import traitlets, requests, ipywidgets, pandas, jinja2, IPython, odo, multipledispatch, blaze, logging, http, sys, json
from coffeetools import coffee
%reload_ext autoreload
%autoreload 2
IPython.display.Javascript( coffee.compile("""
        window.update = ( selection, data, tag = 'div',  classes = [] )->
            selection = selection.selectAll [tag, classes...].join '.'
                .data data
            selection.enter().append tag
            for c in classes
                selection.classed c, true
            selection.exit().remove()
            selection
"""))
get_ipython = ipywidgets.get_ipython

In [28]:
@odo.resource.register( 'gh://.*' )
def github__shorthand_resource( path, *args, **kwargs ):
    """Get user or repository level information."""
    __route__ = 'gh://'
    user = path.split( __route__ )[1].split('/')
    user, repo = user if len(user) == 2 else [user[0],'']
    method = 'repos' if repo else 'users'    
    return jinja2.Template( 'https://api.github.com/{{method}}/{{user}}/{{repo}}').render(**locals()).rstrip('/')

In [29]:
@multipledispatch.dispatch( str, dict, requests.sessions.Session )
def github_request( url, params = {}, session = requests.Session() ):
    """
    Make the URL when a url is passed to the function
    
            req, resp = github_request('tonyfast', {}, requests.Session() )
    """
    if not url.startswith('http'):
        url = odo.resource( 'gh://' + url )
    request = requests.Request('GET', url=url, params=params.copy(), headers={'User-Agent': 'tonyfast'} )
    request.params =  request.params 
    return request, session.send( request.prepare() )

@multipledispatch.dispatch( requests.models.Request, requests.models.Response )
def github_request( requests, response ):
    del requests.params['access_token']
    return requests.prepare(), response.json()

In [124]:
class Projects( traitlets.HasTraits ):
    current = traitlets.List([])
    access_token = ipywidgets.Text('asdfasdf')
    username = ipywidgets.Text('tonyfast')
    columns = ipywidgets.SelectMultiple( options=['<Empty>'] )
    row = ipywidgets.IntSlider( description='Row Index', min=0, max=0 )
    session = requests.Session()
    catalog = {}   
    
    def __init__( self, projects = [], *args, **kwargs ):
        self.env = jinja2.Environment( loader = jinja2.FileSystemLoader('_layouts') )
        for project in projects:
            self.add_project( project )
                
    @traitlets.observe('current')
    def _update_projects(self, change, *args, **kwargs):
        for project in change['new']:            
            if not project in change['old']:
                self._make_request( project, alias=project )

    def _make_request( self, project_or_url, params = {}, alias=None ):     
        recorded = self._record( *github_request( project_or_url, params, self.session ) )
        if alias:
            if isinstance( alias, str ):
                self.catalog[alias] = recorded
        
    def _record( self, request, response ):
        url = request.prepare().url
        if isinstance( url, str ):
            self.catalog[url] = {
                'response': response,
                'time': str(pandas.datetime.now()),
                'params': request.params.copy(),
                'request': request,
                'url': url
            } 
            return self.catalog[url] 

    def more( self, project_or_url ):
        params = {}
        self._make_request( project_or_url, params, self.session )
        
    def add_project( self, project ):
        tmp = self.current.copy()
        tmp.append(project)
        self.current = tmp

In [125]:
projects = Projects(['jupyter/notebook'])
projects.add_project('bokeh/bokeh')
projects.more(projects.catalog['bokeh/bokeh']['response'].json()['events_url'])
projects.catalog['https://api.github.com/repos/bokeh/bokeh/events']['response'].json()

[{'actor': {'avatar_url': 'https://avatars.githubusercontent.com/u/1796208?',
   'gravatar_id': '',
   'id': 1796208,
   'login': 'birdsarah',
   'url': 'https://api.github.com/users/birdsarah'},
  'created_at': '2016-02-17T18:53:47Z',
  'id': '3656179940',
  'org': {'avatar_url': 'https://avatars.githubusercontent.com/u/8440965?',
   'gravatar_id': '',
   'id': 8440965,
   'login': 'bokeh',
   'url': 'https://api.github.com/orgs/bokeh'},
  'payload': {'before': '2922ff55138670ef29d43c5599a6783f51c03d15',
   'commits': [{'author': {'email': 'sarah@bonvaya.com', 'name': 'Sarah Bird'},
     'distinct': True,
     'message': 'Run sauce-connect in background and simplify further',
     'sha': '77cfd3a26d1e0599ee154ec5c4bbec22003ed112',
     'url': 'https://api.github.com/repos/bokeh/bokeh/commits/77cfd3a26d1e0599ee154ec5c4bbec22003ed112'}],
   'distinct_size': 1,
   'head': '77cfd3a26d1e0599ee154ec5c4bbec22003ed112',
   'push_id': 981707567,
   'ref': 'refs/heads/birdsarah/3827_test_with_f

In [134]:
project = ipywidgets.Dropdown( options = projects.current )
project_div = IPython.display.HTML("""<div id='project'></div>""")

def update_project_dashboard( name, old, new ):
    repo_data = projects.catalog[new]['response'].json()
    return IPython.display.display(IPython.display.Javascript( coffee.compile( jinja2.Template("""
    update d3.select('#project'),['{{data['name']}}'], 'h1'
        .text (d)-> d
    """).render(data=repo_data, json=json))))
project._value_changed = update_project_dashboard

IPython.display.display( project, project_div )

In [137]:
catalog = ipywidgets.Dropdown(options=[k for k in projects.catalog.keys() if k.startswith('http')])
catalog_div = IPython.display.HTML("""<div id='catalog'></div>""")

def update_catalog_dashboard( name, old, new ):
    repo_data = projects.catalog[new]['response'].json()
    if not isinstance(repo_data, list ):
        repo_data = [ repo_data ]
    return IPython.display.display(IPython.display.Javascript( coffee.compile( jinja2.Template("""
    console.log 1
    update d3.select('#catalog'),{{json.dumps(data)}}, 'code'
        .text (d)-> JSON.stringify d
    """).render(data=repo_data,json=json))))
catalog._value_changed = update_catalog_dashboard

IPython.display.display( catalog, catalog_div )

<IPython.core.display.Javascript object>

In [50]:
%%file _layouts/table.html
<table id="catalog" class="table"></table>

Overwriting _layouts/table.html


In [51]:
%%file _layouts/update_table.coffee
data = {{df}}
table = d3.select '#catalog'
header = table.selectAll 'tr.header'
    .data [ '', data['columns'] ]
header.enter().append('tr').classed 'header', true
header.each (d)->
    cells = d3.select(@).selectAll('th').data d
    cells.enter().append 'th'
    cells.text (d)-> d
    cells.exit().remove()
header.exit().remove()
value = table.selectAll 'tr.value'
    .data data['data']
value.enter().append('tr').classed 'value', true
value.each (d)->
    cells = d3.select @
        .selectAll 'td'
        .data d
    cells.enter().append 'td'
    cells.text (d)-> JSON.stringify d
    cells.exit().remove()  
value.exit().remove()

Overwriting _layouts/update_table.coffee
